In [1]:
import pandas as pd
import os

# Importando tabelas

In [2]:
df_dados = pd.read_excel('dados.xlsx')
df_fator = pd.read_excel('fatores.xlsx')

lst_meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
lst_meses_dados = ['Dez anterior','Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov']
lst_layout = ['Galpões', 'Complemento','Registro', 'Empresa', 'Tipo de imóvel']

# Gerando Data Frame das diferenças

In [3]:
df_diferença = df_dados[lst_layout]

for m, d in zip(lst_meses, lst_meses_dados):
    dif = df_dados.loc[:, m] - df_dados.loc[:, d]
    df_diferença.loc[:, m] = dif


df_diferença = df_diferença.loc[~df_diferença['Jan'].isna(), :].reset_index(drop=True)

df_fator.loc[3, 'Volume max de água m3'] = 999999


C:\Users\Gabriel\AppData\Local\Temp\ipykernel_1784\2604391656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diferença.loc[:, m] = dif
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_1784\2604391656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diferença.loc[:, m] = dif
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_1784\2604391656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

# Gerando Data Frame dos preços

In [4]:
# As funções definidas a seguir foram somente para organizar a funcionalidade do loop

# def df_taxas_xm(index, m, type):
#     df_temp = df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type), m]
#     return df_temp

# def df_taxas_Mxm(index, m, type):
#     #type foi utilizada fora da função como o tipo de imóvel. No caso do loop, como var
#     df_temp = df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m]
#     return df_temp

# def df_taxas_Mx(index, m, type):
#     df_temp = df_preço.loc[(df_diferença[m] >= df_fator.loc[index,'Volume min de água m3']) & (df_diferença['Tipo de imóvel'] == type), m]
#     return df_temp


df_preço = df_diferença.loc[df_diferença['Galpões'] != 'Poço', lst_layout]

for m in lst_meses:
    
    for type in ['Industrial', 'Residencial']:
        df_preço.loc[(df_diferença[m] < df_fator.loc[0,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type), m] = df_fator.loc[0, f'Fator {type}']
        
        for index in range(1, 4):
            df_preço_temp = df_diferença.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m]  
            df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m] = df_preço_temp * df_fator.loc[index, f'Fator {type}']

df_preço = df_preço.round(2)
# df_preço[lst_meses] = df_preço[lst_meses].map(lambda x: f'R$ {x}')

# TOTAL
num_index = df_preço['Registro'].value_counts().sum()
df_preço.loc[num_index,:] = df_preço.sum(numeric_only=True, axis=0)
df_preço.loc[num_index, 'Galpões'] = 'TOTAL'

# # TOTAL
# num_index = df_preço.index[-1]
# df_preço.loc[num_index + 1,:] = df_preço.sum(numeric_only=True, axis=0)
# df_preço.loc[num_index + 1, 'Galpões'] = 'TOTAL'

df_preço.to_excel('Tabela preços água.xlsx', index=False)





        


# Separando arquivos na pasta compartilhamento

In [5]:
if not os.path.exists(r'compartilhamento'):
    os.mkdir(r'compartilhamento')

# lista de empresas
lst_empresa = df_preço['Empresa'].value_counts().index.tolist()

for comp in lst_empresa:

    # criação de DataFrame do custo $ para cada empresa
    df_preço_temp = df_preço.loc[df_preço['Empresa'] == comp, ['Galpões','Complemento','Registro']+lst_meses]
    df_preço_temp = df_preço_temp.dropna(how='all', axis=1).reset_index(drop=True)
    

    # criação de DataFrame do consumo m3 para cada empresa
    df_diferença_temp = df_diferença.loc[df_diferença['Empresa'] == comp, ['Galpões','Complemento','Registro']+lst_meses]
    df_diferença_temp = df_diferença_temp.dropna(how='all', axis=1).reset_index(drop=True)
    

    # criação de DataFrame do registro m3 para cada empresa
    df_dados_temp = df_dados.loc[df_dados['Empresa'] == comp, ['Galpões','Complemento','Registro']+lst_meses]
    df_dados_temp = df_dados_temp.dropna(how='all', axis=1).reset_index(drop=True)





    # lista com meses completos
    lst_meses_uteis = df_preço_temp.columns.tolist()
    lst_meses_uteis.remove('Galpões')
    try:
        lst_meses_uteis.remove('Complemento')
    except:
        i=0
    lst_meses_uteis.remove('Registro')



    
    # implementação de TOTAL para custo $
    num_index = df_preço_temp['Registro'].value_counts().sum()
    df_preço_temp.loc[num_index,:] = df_preço_temp.sum(numeric_only=True, axis=0)
    df_preço_temp.loc[num_index, 'Galpões'] = 'TOTAL'
    

    # implementação de TOTAL para consumo m3
    num_index = df_diferença_temp['Registro'].value_counts().sum()
    df_diferença_temp.loc[num_index,:] = df_diferença_temp.sum(numeric_only=True, axis=0)
    df_diferença_temp.loc[num_index, 'Galpões'] = 'TOTAL'

    
    # formatação de R$
    df_preço_temp[lst_meses_uteis] = df_preço_temp[lst_meses_uteis].map(lambda x: f'R$ {x}')
    
    # formatação de m3
    df_diferença_temp[lst_meses_uteis] = df_diferença_temp[lst_meses_uteis].map(lambda x: f'{x:.1f} m³')

    # combinando DataFrames
    df_final_temp = pd.concat([df_dados_temp, df_diferença_temp, df_preço_temp], axis=0)


    # geração de arquivo excel
    df_final_temp.to_excel(fr'compartilhamento\{comp}.xlsx', index=False)
    

KeyError: "['Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'] not in index"